# Installing `AllanTools`

"`AllanTools` is a library for calculating Allan Deviation and related time & frequency statistics"

[AllanTools on PyPi](https://pypi.org/project/AllanTools/)

In [ ]:
!pip install allantools

# Using Real IMU Data

Courtesy of Lia, IMU Data is made available. Given the dataset size, the filetype has been converted to HDF5 format via another Python script. The resulting HDF5 was uploaded to Google Drive. The cell below will mount Google Drive to the local directory, giving this notebook access to the data at the specified path.

This process requires authorization from the Google Account running the notebook.

In [ ]:
#gives access to data saved on google drive
from google.colab import drive
drive.mount("/gdrive")

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


Alternatively, local files can be uploaded using the following recipe

In [ ]:
#from google.colab import files
#files.upload()

## Preamble

In [ ]:
#for working with local files
import os
from shutil import rmtree

In [ ]:
#scientific libraries
from allantools import adev, oadev
import pandas as pd
import numpy as np 
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt

In [ ]:
#path to combined dataset in Google Drive
path = "/gdrive/My Drive/Colab Notebooks/IMU Error Research/Datasets/combined_IMU_data.h5"

In [ ]:
#path = "/content/6-Coupled 1.csv"

## Helper functions

The following functions are created to automate common tasks.

Descriptions for each function is given as a docstring.

In [ ]:
def delete_selected_directory(path_to_dir):
    """
    Wrapper for `shutil.rmtree()`
    """
    rmtree(path_to_dir)
    return 

In [ ]:
def log_straight_line(slope, intercept, x):
    """
    Calculate a point on a straight line with desired slope
    """
    return intercept*(x**slope)

In [ ]:
def chunk_number():
    """
    Helper generator to keep track of which chunk the iteration is on
    """
    num=0
    while True:
        yield num
        num += 1

## Data Ingestion

The combined IMU dataset takes ~2 GB of memory. Loading the entire dataset would not leave enough RAM for computation. So, the functions below handle data ingestion by reading **chunks** of data at a time.

Although a functional approach was taken here, future versions of the project will maximize code readability with an OOP approach. 

In [ ]:
#TODO:  Refactor as OOP

In [ ]:
def iter_chunk(path, chunksize, want_column="Sensor"):

    """
    Iterate over dataset and return a timeseries of `want_column`
    """

    #read in data as an interable
    reader = pd.read_hdf(path, iterator=True, chunksize=chunksize)
    #reader = pd.read_csv(path, iterator=True, chunksize=chunksize)

    #keep track of chunk
    gen = chunk_number()

    for chunk in reader:
        #define columns
        chunk.columns = ["Sensor", "Laser"]

        cand_df = pd.DataFrame(chunk)

        #FIXME:  need verifcation that `want_column` is in `chunk.columns`
        results = np.array(cand_df[want_column])

        #calculate timestamps based on row index
        if len(cand_df)==chunksize:
            cand_df["Timestep"] = [(cand_df.index[0]+i)*10e-6 for i in range(0, chunksize)]

        #len(reader)/chunksize has a remainder
        else:
            cand_df["Timestep"] = (cand_df.index[0])*10e-6

        timesteps = np.array(cand_df["Timestep"])

        num = next(gen)

        #generate values with calculated data
        yield timesteps, results

## Data Visualisation

With an efficent method for ingesting data, it's possible to visualize it. The methods below are specific to generating the 2 graphs.

### Real IMU Sensor Data as a Time Series

To create the dataset being explored here, several smaller files were combined. Viewing the combined dataset as a single time series provides a way of checking that the ordering of the samples was maintained during combination.

The IMU signal was sampled every 10 $\mu$s (f$_{s}$=100kHz). So, the x-axis records the elapsed time of the experiment and has units of seconds.

The sample readings (i.e. y-values) are the raw values of the FPGA on-board the IMU. These values are directly proportional to voltage. So, the y-axis is left unitless. In the future, these values could be scaled, but it is not required.


In [ ]:
def plot_iterator(iterator, save_directory, chunksize=1_000, total_runs=10_000, 
                  checkpoint_run=1_000, 
                  title="Real IMU Time Series Run: {}", 
                  filename="REAL_IMU_TIME_SERIES_RUN_{}.png"):
    """
    Plot a dataset as a time series

    The total number of points plotted is given by:
    Total Points = `chunksize`*`total_runs`

    Save a plot after every P points where:
    P = `chunksize`*`checkpoint_run`
    """

    #create a directory to save plots
    try:
        os.mkdir(save_directory)
    #directory already exists
    except FileExistsError:
        pass

    for run, chunk in enumerate(iterator):
        #TODO:  give user control over which values to plot
        #plt.plot(chunk[0], chunk[1], 'r,')
        plt.plot(chunk[0], chunk[1], 'k,')

        plt.xlabel("Elapsed Time (s)")

        if (run<total_runs):
            if (run%checkpoint_run)==0:
                plt.title(title.format(run))
                #TODO:  sanitize `save_directory` and `filename`
                plt.savefig(save_directory+filename.format(run))
        else:
            plt.close()
            break
    return 

In [ ]:
#plot the real data as a time series using `plot_iterator`
chunksize = 100_000
iterator = iter_chunk(path, chunksize=chunksize)
save_directory = "plot_iterator_directory/"


plot_iterator(iterator=iterator, chunksize=chunksize, save_directory=save_directory)

KeyboardInterrupt: ignored

### Allan Deviation of Real IMU Data

The main point of this notebook is to test if `adev()` can reliably reproduce the Allan deviation of a known signal. In the cells below, `plot_adev()` uses `adev()` to calculate the Allan deviation of the combined dataset and plots the results.

Values for a power law relationship and the calculated Allan deviation are plotted simultaneously. The power law will appear as a straight line with a specific slope when plotted on a log-log plot. This straight line will act as a reference for checking the slope of the Allan deviation  curve. 

Here the slope is set to -0.5; the expected slope of the Allan deviation calculated for a white noise sequence. The slope of the reference line can be adjusted, as needed.

In [ ]:
def plot_adev(iterator, rate, save_directory, chunksize=1_000, total_runs=10_000, 
                  checkpoint_run=1_000, 
                  title="Allan Deviation on Real IMU Data Run: {}", 
                  filename="ALLAN_DEVIATION_ON_REAL_IMU_DATA_RUN_{}.png"):
    """
    Calculate the Allan Deviation over the data in `iterator`
    """

    #create a directory to save plots
    try:
        os.mkdir(save_directory)
    #directory already exists
    except FileExistsError:
        pass

    for run, chunk in enumerate(iterator):
        #calculate the allan deviation
        (_, a_plot, _, _) = adev(data=chunk[1], rate=10e5, taus="decade")
        a_plot = np.sqrt(a_plot)

        #FIXME:  shift taus forward by fixed amount with each run without manual calculation
        #manually calculate taus
        powers = np.arange(0, len(a_plot), 1)
        t_plot = np.power(2, powers+(run*len(a_plot)))
        t_plot = t_plot

        """
        elif taus is "decade":  # 1, 2, 4, 10, 20, 40, spacing similar to Stable32
            maxn = np.floor(np.log10(len(data)))
            taus = []
            for k in range(int(maxn+1)):
                taus.append(1.0*(1.0/rate)*pow(10.0, k))
                taus.append(2.0*(1.0/rate)*pow(10.0, k))
                taus.append(4.0*(1.0/rate)*pow(10.0, k))

        maxn = np.floor(np.log10(len(chunk[1])))
        
        t_plot = []
        start = 0
        stop = int(maxn+1)

        if run==0:
            for k in range(start, stop):
                t_plot.append(1.0*(1.0/rate)*pow(10.0, k))
                t_plot.append(2.0*(1.0/rate)*pow(10.0, k))
                t_plot.append(4.0*(1.0/rate)*pow(10.0, k))
        else:
            start+=run*stop
            stop+=run*stop

            for k in range(start, stop):
                t_plot.append(1.0*(1.0/rate)*pow(10.0, k))
                t_plot.append(2.0*(1.0/rate)*pow(10.0, k))
                t_plot.append(4.0*(1.0/rate)*pow(10.0, k))

        t_plot = t_plot[0:len(a_plot)]
        """

        #allan deviation plotting
        plt.loglog(t_plot, a_plot, "kD", label="Allan Deviation")

        #simultaneously plot a straight line (loglog scale)
        #plt.loglog(t_plot, log_straight_line(-0.5, 10e4, t_plot), "r8", label="Line $\\nabla=-0.5$")
        plt.xlim(xmax=np.array([10**20], dtype=float))

        plt.xlabel("$\\tau$")
        plt.ylabel("$\sigma(\\tau)$")
        #plt.legend()

        if (run<total_runs):
            if (run%checkpoint_run)==0:
                plt.title(title.format(run))
                #TODO:  sanitize `save_directory` and `filename`
                plt.savefig(save_directory+filename.format(run))
        else:
            plt.close()
            break
    return 

### Consequence of Changing Chunksize

The data ingestion code (above) always has access to the timesteps of the current chunk. Once processing is finished with one chunk, it is discarded and the next chunk is loaded in. As a result, the chunk timestep is also reset $\rightarrow$ 0 to length(chunk). While `adev()` will automatically generate *$\tau$* values for a given chunk of data, stitching the chunks together has proven difficult.

Instead of manually building the time axis, the chunksize can be gradually increased. Gradually increasing the chunksize has the same effect as increasing the experiment's measurement time, while also pushing the work of computing $\tau$ onto `adev()`.

In [ ]:
#plot Allan deviation using `plot_adev`
chunksize = 1_000_000_000
checkpoint_run = 1000
total_runs = 100
iterator = iter_chunk(path, chunksize=chunksize)

save_directory = "bias_instability_directory/"


title = f"Allan Deviation of REAL IMU\nChunksize: {chunksize}"
filename = "SEARCHING_FOR_BIAS_INSTABILITY_RUN_{}.png"

plot_adev(iterator=iterator, rate=100000, chunksize=chunksize, checkpoint_run=checkpoint_run, total_runs=total_runs, save_directory=save_directory, title=title, filename=filename)

## Running the Notebook

In this first version of the notebook, the generated plots are saved in a local runtime directory. However, if unforseen errors do arise deleting the plot directories and restarting the notebook runtime may fix these errors.